## Example of training/testing using deterministic models with pytorch-lightning

In [1]:
from pathlib import Path
from types import SimpleNamespace
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path: # To be able to import files 
    sys.path.append(module_path) # without installing the project

from bayesrul.cmapss.dataset import CMAPSSDataModule
from bayesrul.cmapss.models import CMAPSSModel, get_checkpoint, TBLogger

/home/arthur/.conda/envs/bayesrul/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
args = SimpleNamespace(
    data_path="../data/cmapss/",
    out_path="../results/cmapss/",
    scn="dnn_ptl",
    net="linear",
    lr=1e-3
)
checkpoint_dir = Path(f"{args.out_path}/{args.scn}/checkpoints/{args.net}2")
logger = TBLogger(
    f"{args.out_path}/{args.scn}/lightning_logs/{args.net}",
    default_hp_metric=False,
)

In [3]:
# Just to illustrate. To properly train see scripts/cmapss_training.py
data = CMAPSSDataModule(args.data_path, batch_size=10000)
dnn = CMAPSSModel(data.win_length, data.n_features, args.net)

checkpoint_file = get_checkpoint(checkpoint_dir)
monitor = "loss/val"
checkpoint_callback = ModelCheckpoint(dirpath=checkpoint_dir, monitor=monitor)
earlystopping_callback = EarlyStopping(monitor=monitor, patience=10)

trainer = pl.Trainer(
    gpus=[0],
    max_epochs=1000,
    log_every_n_steps=2,
    logger=logger,
    callbacks=[
        checkpoint_callback,
        earlystopping_callback,
    ],
)
trainer.fit(dnn, data, ckpt_path=checkpoint_file)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ../results/cmapss//dnn_ptl/lightning_logs/linear/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name | Type   | Params
--------------------------------
0 | net  | Linear | 45.6 K
--------------------------------
45.6 K    Trainable params
0         Non-trainable params
45.6 K    Total params
0.182     Total estimated model params size (MB)


Epoch 72: 100%|██████████| 3/3 [00:56<00:00, 18.73s/it, loss=1.93e+03, v_num=0]


In [4]:
data = CMAPSSDataModule(args.data_path, batch_size=1000)
dnn = CMAPSSModel.load_from_checkpoint(get_checkpoint(checkpoint_dir))
trainer = pl.Trainer(gpus=[0], log_every_n_steps=10, logger=logger)
trainer.test(dnn, data, verbose=False)

/home/arthur/.conda/envs/bayesrul/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Testing DataLoader 0: 100%|██████████| 12/12 [00:00<00:00, 36.68it/s]


[{'loss/test': 26622.8984375}]

In [5]:
dl = data.train_dataloader()
print(f"Win length {data.win_length}, n_features {data.n_features}")
ex = next(iter(dl))

Win length 18, n_features 14
